In [4]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [5]:
# mel spectogram converts audio to visual form
# uses librosa library

def extract_features(audio_data):
  
       wav_data = io.BytesIO(audio_data.get_wav_data())
       y, sr_rate = librosa.load(wav_data, sr=22050)
        
       mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
        # Mel-frequency cepstral coefficients - power spectrum of sound
       chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)
        # chromagram -> vector representation of audio, representing pitch on chromatic scale
       mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate).T, axis=0)
        # compute mel-scaled spectrogram
        
       features = np.hstack([mfccs, chroma, mel])
       return features  
    
 

In [6]:
r = sr.Recognizer()

while True:
    try:
        with sr.Microphone() as source:
            print("\nListening...")
            r.adjust_for_ambient_noise(source, duration=1) 
            # convert audio to WAV format and get features
            audio_data_as_file = io.ByFtesIO(audio.get_wav_data()) 
            features = extract_features_from_buffer(audio_data_as_file)
            
            if features is not None: 
                print(f"Features extracted! Shape: {features.shape}")
            
            text = r.recognize_google(audio).lower()
            print("You said:", text)
            
            if "exit" in text:
                print("Exiting program...")
                break
            
    except sr.UnknownValueError:
        print("Could not understand audio")
    except Exception as e:
        print(f"Loop Error: {e}")
        break
        
        

NameError: name 'sr' is not defined